In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random , logstash , logging , pymongo



In [ ]:


host = '192.168.114.130'
client = "mongodb://" + host + ":27017/"
# 多執行續下 可能被 dead lock 
myclient = pymongo.MongoClient(client, connect=False)
mydb = myclient["mydatabase"]
mycol = mydb["Mobile01"]


In [ ]:
# pipeline 
logger = logging.getLogger('python-logstash-logger')
logger.setLevel(logging.INFO)
host_demo = "logstash"
logger.addHandler(logstash.TCPLogstashHandler(host_demo,5959))




In [2]:
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}



def get_connect(url):
    i = 0
    
    while True :
        
        try :
            resp = requests.get(url,headers=header,proxies=proxylist[i])          
            resp.encoding = "utf-8"
            time.sleep(random.randint(1,4))
           
            
        except requests.exceptions.ConnectionError :             
            #logger.warning("Proxy Error",extra={"proxy":proxylist[i]})
            
            i = (i + 1) % 6
            continue

            
            
        if resp : 
            if resp.status_code == 200 :
                code = "HTTP response code = " + str(resp.status_code)
                #logger.info(code,extra={"url" : url, "time_use" : resp.elapsed.total_seconds()})
                #print(proxylist[i])
                break         
        
        
    return resp.text 
    

In [3]:
url = "https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&pid=2&p=1"

soup = BeautifulSoup(get_connect(url))

In [4]:
table = soup.find_all("tr")
table[0]

<tr>
<td class="icon" width="200"><a href="waypointdetail.php?id=32535"><img border="0" height="150" src="//attach.mobile01.com/waypoint/mobile01-dba7b25a2157b4155d29721a91370806.jpg" width="200"/></a></td>
<td class="detail"><p class="title"><a class="rb16" href="waypointdetail.php?id=32535">臺北市 北投區 溫泉博物館 — 鬱鬱蔥蔥中的紅磚建築，英式建築中的日式溫泉鄉</a></p>
<p class="desc">北投溫泉博物館，在日治時期為「北投公共浴場」，完工於1913年6月，距今已有105年。 建築本體為雙層和洋折衷風格建築，外觀仿英式建築，而內部則部分為和式空間，最初的用途就是溫泉浴室。 日本人愛泡溫泉舉世聞名，在發現北投的優質磺泉後，即儘...</p>
<p class="info">景點分類: <a href="waypointlist.php?pid=2">台北市</a> - 風景 - 博物館或紀念館<br/>
                    發佈日期: 2019-01-11 <a class="bk12u" href="userinfo.php?id=537205">老狗5550</a><br/>
                    無線網路: 有<br/>
</p></td>
</tr>

In [38]:
# 取得列表內容
for t in table:
    title = t.find("p", class_="title").text
    href = "https://www.mobile01.com/" + t.find("p", class_="title").find("a")["href"]
    sp = t.find("p", class_="info").text.split("-")[1]
    date = t.find("p", class_="info").text.split(":")[2].split(" ")[1]
    
        
    

In [61]:
href = "https://www.mobile01.com/waypointdetail.php?id=32523"
soup = BeautifulSoup(get_connect(href))

In [62]:
# 文章本文
content = soup.find("div", class_="single-post-content").text.replace("\r","").replace("\n","").replace(" ","")
img_url = []
imgs = soup.find("div", class_="single-post-content").find_all("img")
for img in imgs:
        img_url.append(img["data-src"])
author = soup.find("div", class_="panel note sidebar-authur").find("a").text
article_hit = soup.find_all("div",class_="panel note ")[1].find_all("li")[0].text.split(":")[-1]
rate = soup.find_all("div",class_="panel note ")[1].find_all("li")[1].text.split(":")[-1]

In [81]:
# 餐廳資訊
len(soup.find(id="info_form").find_all("div")[8].find_all("img"))



5

In [84]:
soup.find(id="info_form").find_all("div")[1].text.replace("\n","")

'地址:士林官邸公園'